In [1]:
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

In [4]:
data=pd.read_csv('/content/Online Retail (1).csv',encoding='ISO-8859-1')

In [5]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26:00,3.39,17850.0,United Kingdom


In [6]:
data['total_bill']=data['Quantity']*data['UnitPrice']

In [7]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_bill
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26:00,3.39,17850.0,United Kingdom,20.34


In [8]:
data['Description']=data['Description'].str.strip()
#remove spaces from data

In [9]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
total_bill          0
dtype: int64

In [10]:
data.dropna(inplace=True)

In [11]:
data.drop_duplicates(inplace=True)

In [12]:
data.shape

(401604, 9)

In [13]:
d1=data.groupby('CustomerID').agg({'InvoiceNo':'count',
                                  'Quantity':'sum',
                                   'total_bill':'sum'})

In [14]:
d1.columns=['invoice_frequency','total_quantity','total_bill']

In [15]:
d1.head()

,invoice_frequency,total_quantity,total_bill
CustomerID,,,
12346.0,2,0,0.00
12347.0,182,2458,4310.00
12348.0,31,2341,1797.24
12349.0,73,631,1757.55
12350.0,17,197,334.40


In [16]:
d1['avg_bill']=d1['total_bill']/d1['invoice_frequency']

In [17]:
data['InvoiceDate']=pd.to_datetime(data['InvoiceDate'])

In [18]:
latest_date=data['InvoiceDate'].max()

In [19]:
data['day_gap']=(latest_date-data['InvoiceDate'])

In [20]:
d2=data.groupby('CustomerID')['day_gap'].min().reset_index()

In [21]:
d2['day_gap']=d2['day_gap'].dt.days

In [22]:
d12=pd.merge(d1,d2,on='CustomerID',how='inner')

In [23]:
d12.head()

,CustomerID,invoice_frequency,total_quantity,total_bill,avg_bill,day_gap
0,12346.0,2,0,0.00,0.000000,325
1,12347.0,182,2458,4310.00,23.681319,1
2,12348.0,31,2341,1797.24,57.975484,74
3,12349.0,73,631,1757.55,24.076027,18
4,12350.0,17,197,334.40,19.670588,309


In [24]:
d3=data.groupby('CustomerID')['Country'].unique().reset_index()

In [25]:
d3.head()

,CustomerID,Country
0,12346.0,[United Kingdom]
1,12347.0,[Iceland]
2,12348.0,[Finland]
3,12349.0,[Italy]
4,12350.0,[Norway]


In [26]:
x=list(d3['Country'].values)

In [27]:
m=[]
for i in range(len(x)):
   c=str(x[i][0])
   m.append(c)

In [28]:
d3['country']=m

In [29]:
d3.drop('Country',axis=1,inplace=True)

In [30]:
d3.head()

,CustomerID,country
0,12346.0,United Kingdom
1,12347.0,Iceland
2,12348.0,Finland
3,12349.0,Italy
4,12350.0,Norway


In [31]:
df=pd.merge(d12,d3,on='CustomerID',how='inner')

In [32]:
df.head()

,CustomerID,invoice_frequency,total_quantity,total_bill,avg_bill,day_gap,country
0,12346.0,2,0,0.00,0.000000,325,United Kingdom
1,12347.0,182,2458,4310.00,23.681319,1,Iceland
2,12348.0,31,2341,1797.24,57.975484,74,Finland
3,12349.0,73,631,1757.55,24.076027,18,Italy
4,12350.0,17,197,334.40,19.670588,309,Norway


In [33]:
from sklearn.preprocessing import OrdinalEncoder

In [34]:
c=list(pd.unique(df['country']))

In [35]:
o=OrdinalEncoder(categories=[c])
df['country_code']=o.fit_transform(df[['country']])
df.drop('country',axis=1,inplace=True)
df.head()

,CustomerID,invoice_frequency,total_quantity,total_bill,avg_bill,day_gap,country_code
0,12346.0,2,0,0.00,0.000000,325,0.0
1,12347.0,182,2458,4310.00,23.681319,1,1.0
2,12348.0,31,2341,1797.24,57.975484,74,2.0
3,12349.0,73,631,1757.55,24.076027,18,3.0
4,12350.0,17,197,334.40,19.670588,309,4.0


In [272]:
df_clean=df.drop(['CustomerID','avg_bill','country_code','total_quantity'],axis=1)

In [273]:
df_clean.head()

,invoice_frequency,total_bill,day_gap
0,2,0.00,325
1,182,4310.00,1
2,31,1797.24,74
3,73,1757.55,18
4,17,334.40,309


In [274]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.metrics import silhouette_score

In [275]:
df_final=df_clean[df_clean['total_bill']>0]

In [276]:
df_final.head()

,invoice_frequency,total_bill,day_gap
1,182,4310.00,1
2,31,1797.24,74
3,73,1757.55,18
4,17,334.40,309
5,95,1545.41,35


In [277]:
df_final.shape

(4322, 3)

In [278]:
!pip install optuna

In [279]:
import optuna

In [280]:
s=StandardScaler()
v=s.fit_transform(df_final)
d_sc=pd.DataFrame(v,columns=df_clean.columns)
d_sc.head()

,invoice_frequency,total_bill,day_gap
0,0.387049,0.289474,-0.891260
1,-0.268530,-0.014677,-0.154795
2,-0.086184,-0.019482,-0.719755
3,-0.329313,-0.191744,2.216019
4,0.009331,-0.045160,-0.548249


In [281]:
d_sc.describe().T

,count,mean,std,min,25%,50%,75%,max
invoice_frequency,4322.0,6.576055e-18,1.000116,-0.398778,-0.324971,-0.220773,0.035381,33.513359
total_bill,4322.0,3.288027e-18,1.000116,-0.232221,-0.195835,-0.153140,-0.035633,33.597941
day_gap,4322.0,-4.767640e-17,1.000116,-0.901349,-0.739932,-0.412053,0.480785,2.861687


In [301]:
def objective(trial):
    eps=trial.suggest_float('eps',.4,.5,log=True)
    min_samples=trial.suggest_int('min_samples',3,6)
    model=DBSCAN(eps=eps,min_samples=min_samples)
    model.fit(d_sc)
    if len(set(model.labels_)) > 1:
        silhouette = silhouette_score(d_sc,model.labels_)
    else:
        silhouette = -1
    return silhouette

In [302]:
study=optuna.create_study(direction='maximize')

[I 2024-05-11 08:43:00,007] A new study created in memory with name: no-name-26355b95-6e5c-4482-9505-138cfb3d594a


In [ ]:
study.optimize(objective,n_trials=25)

In [304]:
study.best_params

{'eps': 0.4853076995155518, 'min_samples': 6}

In [305]:
study.best_value

0.7072837713712313

In [306]:
db=DBSCAN(eps=.485,min_samples=6)
label=db.fit_predict(d_sc)

In [307]:
df_final['label']=label
df_final['label'].value_counts()

label
 0    4264
-1      52
 1       6
Name: count, dtype: int64

In [308]:
df_final.drop('label',axis=1,inplace=True)